### ***Python Code***

In [1]:
pip install pulp pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 14.4 MB/s eta 0:00:00


In [2]:
import pulp

# Define model
model = pulp.LpProblem("Brendamore_Sports_Production", pulp.LpMinimize)

# Define data
football_demand = [15000, 25000, 20000, 5000, 2500, 5000]
soccer_demand = [10000, 15000, 10000, 5000, 5000, 7500]
football_prod_cost = [13.80, 13.90, 12.95, 12.60, 12.55, 12.70]
soccer_prod_cost = [10.85, 10.55, 10.50, 10.50, 10.55, 10.00]
football_hold_cost = [0.69, 0.70, 0.65, 0.63, 0.63, 0.64]
soccer_hold_cost = [0.54, 0.53, 0.53, 0.53, 0.53, 0.50]
initial_inventory_football = 7000
initial_inventory_soccer = 5000
ending_inventory_required_football = 3000
ending_inventory_required_soccer = 3000
max_production_capacity = 32000
max_inventory_capacity = 20000

# Decision variables
x_f = [pulp.LpVariable(f'x_f_{t}', lowBound=0) for t in range(6)]
x_s = [pulp.LpVariable(f'x_s_{t}', lowBound=0) for t in range(6)]
h_f = [pulp.LpVariable(f'h_f_{t}', lowBound=0) for t in range(6)]
h_s = [pulp.LpVariable(f'h_s_{t}', lowBound=0) for t in range(6)]

# Objective function: Minimize production and holding costs
model += pulp.lpSum([
    football_prod_cost[t] * x_f[t] + soccer_prod_cost[t] * x_s[t] +
    football_hold_cost[t] * h_f[t] + soccer_hold_cost[t] * h_s[t]
    for t in range(6)
])

# Constraints
# Initial inventory constraints
model += h_f[0] == initial_inventory_football + x_f[0] - football_demand[0]
model += h_s[0] == initial_inventory_soccer + x_s[0] - soccer_demand[0]

# Inventory balance constraints for months 2 to 6
for t in range(1, 6):
    model += h_f[t] == h_f[t-1] + x_f[t] - football_demand[t]
    model += h_s[t] == h_s[t-1] + x_s[t] - soccer_demand[t]

# Ending inventory requirements
model += h_f[5] >= ending_inventory_required_football
model += h_s[5] >= ending_inventory_required_soccer

# Production capacity constraints
for t in range(6):
    model += x_f[t] + x_s[t] <= max_production_capacity

# Inventory capacity constraints
for t in range(6):
    model += h_f[t] + h_s[t] <= max_inventory_capacity

# Solve model
model.solve()

# Output results
for t in range(6):
    print(f'Month {t+1} - Footballs Produced: {x_f[t].varValue}, Soccer Balls Produced: {x_s[t].varValue}')
    print(f'Month {t+1} - Footballs Inventory: {h_f[t].varValue}, Soccer Balls Inventory: {h_s[t].varValue}')


Month 1 - Footballs Produced: 16000.0, Soccer Balls Produced: 5000.0
Month 1 - Footballs Inventory: 8000.0, Soccer Balls Inventory: 0.0
Month 2 - Footballs Produced: 17000.0, Soccer Balls Produced: 15000.0
Month 2 - Footballs Inventory: 0.0, Soccer Balls Inventory: 0.0
Month 3 - Footballs Produced: 20000.0, Soccer Balls Produced: 10000.0
Month 3 - Footballs Inventory: 0.0, Soccer Balls Inventory: 0.0
Month 4 - Footballs Produced: 5000.0, Soccer Balls Produced: 5000.0
Month 4 - Footballs Inventory: 0.0, Soccer Balls Inventory: 0.0
Month 5 - Footballs Produced: 2500.0, Soccer Balls Produced: 5000.0
Month 5 - Footballs Inventory: 0.0, Soccer Balls Inventory: 0.0
Month 6 - Footballs Produced: 8000.0, Soccer Balls Produced: 10500.0
Month 6 - Footballs Inventory: 3000.0, Soccer Balls Inventory: 3000.0


### ***AMPL Code***

In [3]:
# Install dependencies
!pip install -q amplpy ampltools

# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# Instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.4 MB/s eta 0:00:00
Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import pandas to read Excel data
import pandas as pd

# Define the file path
file_path = '/content/drive/MyDrive/Colab Notebooks/Brendamore.xlsx'

Mounted at /content/drive


In [5]:
# Read data from the same sheet in the Excel file
data = pd.read_excel(file_path, sheet_name="Data", index_col=0)

# Assuming the structure of the sheet is as follows:
# The first rows contain football and soccer demand, production cost, and holding cost
# Let's display the data first to verify the structure
print(data)

                                           Football Demand Forecast  \
Month                                                                 
1                                                             15000   
2                                                             25000   
3                                                             20000   
4                                                              5000   
5                                                              2500   
6                                                              5000   
NaN                                                             NaN   
NaN                                                       Footballs   
Current Inventory                                              7000   
Ending Inventory required (end of month 6)                     3000   

                                           Production Cost ($ per football)  \
Month                                                               

In [7]:
# Extract the necessary data:
# Columns for Football Demand, Soccer Demand, Production Cost, and Holding Cost
football_demand = data['Football Demand Forecast'].to_dict()
soccer_demand = data['Soccer Ball Demand Forecast'].to_dict()
football_prod_cost = data['Production Cost ($ per football)'].to_dict()
soccer_prod_cost = data['Production Cost ($ per soccer ball)'].to_dict()
football_hold_cost = data['Holding Cost ($ per football)'].to_dict()
soccer_hold_cost = data['Holding Cost ($ per soccer ball)'].to_dict()

# Combine data into dictionaries compatible with AMPL model
months = list(data.index)
products = ['Football', 'Soccer']

# Clean the months and products data by dropping NaN or unwanted entries
# Clean the months and products data by dropping NaN or unwanted entries
months = [int(month) for month in data.index if pd.notna(month) and str(month).isdigit()]
products = ['Football', 'Soccer']

# Create a combined demand dictionary
demand = {
    ('Football', t): football_demand[t] for t in months
}
demand.update({
    ('Soccer', t): soccer_demand[t] for t in months
})

# Create a combined production cost dictionary
prod_cost = {
    ('Football', t): football_prod_cost[t] for t in months
}
prod_cost.update({
    ('Soccer', t): soccer_prod_cost[t] for t in months
})

# Create a combined holding cost dictionary
hold_cost = {
    ('Football', t): football_hold_cost[t] for t in months
}
hold_cost.update({
    ('Soccer', t): soccer_hold_cost[t] for t in months
})

# Verify the extracted data
print("Demand:", demand)
print("Production Cost:", prod_cost)
print("Holding Cost:", hold_cost)

# Define other parameters (these can stay as they are)
initial_inventory = {"Football": 7000, "Soccer": 5000}
ending_inventory_required = {"Football": 3000, "Soccer": 3000}

prod_capacity = 32000
inv_capacity = 20000


Demand: {('Football', 1): 15000, ('Football', 2): 25000, ('Football', 3): 20000, ('Football', 4): 5000, ('Football', 5): 2500, ('Football', 6): 5000, ('Soccer', 1): 10000, ('Soccer', 2): 15000, ('Soccer', 3): 10000, ('Soccer', 4): 5000, ('Soccer', 5): 5000, ('Soccer', 6): 7500}
Production Cost: {('Football', 1): 13.8, ('Football', 2): 13.9, ('Football', 3): 12.95, ('Football', 4): 12.6, ('Football', 5): 12.55, ('Football', 6): 12.7, ('Soccer', 1): 10.85, ('Soccer', 2): 10.55, ('Soccer', 3): 10.5, ('Soccer', 4): 10.5, ('Soccer', 5): 10.55, ('Soccer', 6): 10}
Holding Cost: {('Football', 1): 0.6900000000000001, ('Football', 2): 0.6950000000000001, ('Football', 3): 0.6475, ('Football', 4): 0.63, ('Football', 5): 0.6275000000000001, ('Football', 6): 0.635, ('Soccer', 1): 0.5425, ('Soccer', 2): 0.5275000000000001, ('Soccer', 3): 0.525, ('Soccer', 4): 0.525, ('Soccer', 5): 0.5275000000000001, ('Soccer', 6): 0.5}


In [8]:
# Define the model in AMPL
ampl.eval('''

reset;

set MONTHS;  # Set of months
set PRODUCTS;  # Set of products (Football and Soccer Balls)

param demand {PRODUCTS, MONTHS};  # Demand for each product in each month
param prod_cost {PRODUCTS, MONTHS};  # Production cost for each product in each month
param hold_cost {PRODUCTS, MONTHS};  # Holding cost for each product in each month
param initial_inventory {PRODUCTS};  # Initial inventory of products
param ending_inventory_required {PRODUCTS};  # Required ending inventory of products
param prod_capacity;  # Total production capacity per month
param inv_capacity;  # Total inventory capacity

var x {PRODUCTS, MONTHS} >= 0;  # Production decision variables
var h {PRODUCTS, MONTHS} >= 0;  # Inventory decision variables

minimize Total_Cost:
    sum {p in PRODUCTS, t in MONTHS} (prod_cost[p,t] * x[p,t] + hold_cost[p,t] * h[p,t]);

# First month demand fulfillment (use initial inventory)
subject to First_Month_Demand_Fulfillment {p in PRODUCTS}:
    initial_inventory[p] + x[p,1] - demand[p,1] = h[p,1];

# Subsequent months demand fulfillment
subject to Subsequent_Months_Demand_Fulfillment {p in PRODUCTS, t in MONTHS: t > 1}:
    h[p,t-1] + x[p,t] - demand[p,t] = h[p,t];

subject to Production_Capacity {t in MONTHS}:
    sum {p in PRODUCTS} x[p,t] <= prod_capacity;

subject to Inventory_Capacity {t in MONTHS}:
    sum {p in PRODUCTS} h[p,t] <= inv_capacity;

subject to Ending_Inventory {p in PRODUCTS}:
    h[p,card(MONTHS)] >= ending_inventory_required[p];

''')


In [9]:
# Provide data to AMPL
ampl.set['MONTHS'] = months
ampl.set['PRODUCTS'] = products

ampl.param['demand'] = demand
ampl.param['prod_cost'] = prod_cost
ampl.param['hold_cost'] = hold_cost
ampl.param['initial_inventory'] = initial_inventory
ampl.param['ending_inventory_required'] = ending_inventory_required

ampl.param['prod_capacity'] = prod_capacity
ampl.param['inv_capacity'] = inv_capacity


In [10]:
# Use ampl.expand to confirm AMPL model syntax is working
ampl.eval('''expand;''')

# Set solver and solve the model
ampl.setOption('solver', 'cbc')

# Solve the model
ampl.solve()


minimize Total_Cost:
	13.8*x['Football',1] + 13.9*x['Football',2] + 12.95*x['Football',3] + 
	12.6*x['Football',4] + 12.55*x['Football',5] + 12.7*x['Football',6] + 
	10.85*x['Soccer',1] + 10.55*x['Soccer',2] + 10.5*x['Soccer',3] + 
	10.5*x['Soccer',4] + 10.55*x['Soccer',5] + 10*x['Soccer',6] + 
	0.69*h['Football',1] + 0.695*h['Football',2] + 0.6475*h['Football',3]
	 + 0.63*h['Football',4] + 0.6275*h['Football',5] + 
	0.635*h['Football',6] + 0.5425*h['Soccer',1] + 0.5275*h['Soccer',2] + 
	0.525*h['Soccer',3] + 0.525*h['Soccer',4] + 0.5275*h['Soccer',5] + 
	0.5*h['Soccer',6];

subject to First_Month_Demand_Fulfillment[
	'Football']:
	x['Football',1] - h['Football',1] = 8000;

subject to First_Month_Demand_Fulfillment[
	'Soccer']:
	x['Soccer',1] - h['Soccer',1] = 5000;

subject to Subsequent_Months_Demand_Fulfillment
	['Football',2]:
	x['Football',2] + h['Football',1] - h['Football',2] = 25000;

subject to Subsequent_Months_Demand_Fulfillment
	['Football',3]:
	x['Football',3] + h['Footbal

In [11]:
# Print results
obj = ampl.get_objective('Total_Cost')
print("\n", "Total cost is: ", obj.get().value(), "\n")

print("Production Plan (x):")
ampl.display('x');

print("Inventory Levels (h):")
ampl.display('h');



 Total cost is:  1448750.0 

Production Plan (x):
x :=
Football 1   16000
Football 2   17000
Football 3   20000
Football 4    5000
Football 5    2500
Football 6    8000
Soccer   1    5000
Soccer   2   15000
Soccer   3   10000
Soccer   4    5000
Soccer   5    5000
Soccer   6   10500
;

Inventory Levels (h):
h :=
Football 1   8000
Football 2      0
Football 3      0
Football 4      0
Football 5      0
Football 6   3000
Soccer   1      0
Soccer   2      0
Soccer   3      0
Soccer   4      0
Soccer   5      0
Soccer   6   3000
;

